#第二周作业-优化代码#


In [55]:
%load_ext Cython


The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [56]:
import numpy as np
import pandas as pd

y = np.random.randint(2,size=(5000,1))
x = np.random.randint(10,size=(5000,1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
#print(data)


In [57]:
def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result


In [58]:
%%timeit

target_mean_v1(data, 'y', 'x')


1 loop, best of 3: 22.4 s per loop


In [59]:
def target_mean_v2(data,y_name,x_name):
  result = np.zeros(data.shape[0])
  v_value  = data.shape[0]
  value_dict = dict()
  count_dict = dict()
  for i in range(v_value):
    if data.loc[i,x_name] not in value_dict.keys():
      value_dict[data.loc[i,x_name]] = data.loc[i,y_name]
      count_dict[data.loc[i,x_name]] = 1
    else:
      value_dict[data.loc[i,x_name]] += data.loc[i,y_name]
      count_dict[data.loc[i,x_name]] +=1
  for i in range(v_value):
    result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
  return result




In [60]:
%%timeit

target_mean_v2(data, 'y', 'x')

1 loop, best of 3: 266 ms per loop


In [61]:
%%cython -a
import numpy as np
def target_mean_v3(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    v_value = data.shape[0]
    for i in range(v_value):
      Xname, Yname = data.loc[i, x_name], data.loc[i, y_name]
      if Xname not in value_dict.keys():
          value_dict[Xname] = Yname
          count_dict[Xname] = 1
      else:
          value_dict[Xname] += Yname
          count_dict[Xname] += 1
    for i in range(v_value):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [62]:
%%timeit

target_mean_v3(data, 'y', 'x')

1 loop, best of 3: 191 ms per loop


In [72]:

%%cython -a
import cython
import numpy as np
cimport numpy as cnp


cpdef cnp.ndarray[double] target_mean_v4(cnp.ndarray[long, ndim=2] data):
    cdef int length = data.shape[0]
    cdef cnp.ndarray[double] result = np.zeros(length)
    cdef dict value_dict = {}
    cdef dict count_dict = {}
    cdef long x_value
    cdef long y_valuez
    cdef unsigned int i
    for i in range(length):
        x_value = data[i, 1]
        y_value = data[i, 0]
        if x_value not in value_dict.keys():
            value_dict[x_value] = y_value
            count_dict[x_value] = 1
        else:
            value_dict[x_value] += y_value
            count_dict[x_value] += 1
    for i in range(length):
        x_value = data[i, 1]
        result[i] = (value_dict[x_value] - data[i, 0]) / (count_dict[x_value] - 1)
    return result

In [73]:
%%timeit

_data = np.concatenate([y, x], axis=1)
target_mean_v4(_data)

1000 loops, best of 3: 1.15 ms per loop


In [75]:
%%cython -a
import cython
import numpy as np
cimport numpy as cnp
from cython.parallel import prange


@cython.boundscheck(False)
@cython.wraparound(False)
cpdef cnp.ndarray[double] target_mean_v5(cnp.ndarray[long, ndim=2] data):
    cdef int[:] x = data[:,1].astype(np.intc)
    cdef int[:] y = data[:,0].astype(np.intc)
    cdef int length = data.shape[0]
    cdef cnp.ndarray[double] result = np.zeros(length)
    cdef int[:] value_dict = np.zeros(length).astype(np.intc)
    cdef int[:] count_dict = np.zeros(length).astype(np.intc)
    cdef int i
    for i in prange(length, nogil=True):
        value_dict[x[i]] += y[i]
        count_dict[x[i]] += 1
    for i in prange(length, nogil=True):
        result[i] = (value_dict[x[i]] - y[i]) / (count_dict[x[i]] - 1)
    return result

In [76]:
%%timeit

_data = np.concatenate([y, x], axis=1)
target_mean_v5(_data)

The slowest run took 20.08 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 54.6 µs per loop
